In [11]:
# %% [markdown]
# # Solana Trader ROI Dashboard
# Build a dashboard to rank Solana traders by ROI for a user-selected list of tokens.
# Metrics: Portfolio Net Worth, Total PnL, Realized/Unrealized PnL, ROI, Num_of_tokens_in_profit

# %%
import os
import aiohttp
import asyncio
import pandas as pd
from datetime import datetime

# Load API keys
HELIUS_API_KEY = os.getenv("HELIUS_API_KEY")
BIRDEYE_API_KEY = os.getenv("BIRDEYE_API_KEY")

HELIUS_BASE = f"https://api.helius.xyz/v0"
BIRDEYE_BASE = "https://public-api.birdeye.so"

headers = {"X-API-KEY": BIRDEYE_API_KEY}

# %% [markdown]
# ## Birdeye Helpers (Historical & Current Prices)

# %%
async def get_historical_price(session, token_mint, timestamp):
    """Fetch historical token price (USD) at a given timestamp."""
    url = f"{BIRDEYE_BASE}/defi/history_price"
    params = {"address": token_mint, "time": timestamp}
    async with session.get(url, headers=headers, params=params) as resp:
        data = await resp.json()
        return data.get("data", {}).get("value", 0)

async def get_current_price(session, token_mint):
    """Fetch current token price (USD)."""
    url = f"{BIRDEYE_BASE}/defi/price"
    params = {"address": token_mint}
    async with session.get(url, headers=headers, params=params) as resp:
        data = await resp.json()
        return data.get("data", {}).get("value", 0)

# %% [markdown]
# ## Helius Helpers (Transactions)

# %%
async def get_token_transfers(wallet):
    """Fetch all token transfers for a wallet from Helius."""
    url = f"{HELIUS_BASE}/addresses/{wallet}/transactions"
    params = {"api-key": HELIUS_API_KEY, "limit": 1000}
    async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as resp:
            return await resp.json()

# %% [markdown]
# ## Trader Analysis

# %%
async def analyze_wallet(wallet, token_list):
    """Analyze one wallet for all metrics."""
    async with aiohttp.ClientSession() as session:
        transactions = await get_token_transfers(wallet)
        
        trades = []
        for tx in transactions:
            events = tx.get("events", [])
            for ev in events:
                if ev.get("type") == "TRANSFER" and ev["token"]["mint"] in token_list:
                    mint = ev["token"]["mint"]
                    amount = float(ev["token"]["amount"])
                    slot_time = int(tx["timestamp"])
                    
                    # Get historical price
                    price = await get_historical_price(session, mint, slot_time)
                    usd_value = price * amount
                    
                    trades.append({
                        "wallet": wallet,
                        "mint": mint,
                        "amount": amount,
                        "timestamp": slot_time,
                        "price_at_trade": price,
                        "value_usd": usd_value,
                        "direction": "in" if ev["token"]["toUserAccount"] == wallet else "out"
                    })
        
        df = pd.DataFrame(trades)
        if df.empty:
            return pd.DataFrame([{
                "wallet": wallet, "portfolio_value": 0, "realized_pnl": 0,
                "unrealized_pnl": 0, "total_pnl": 0, "roi": 0,
                "num_tokens_in_profit": 0
            }])
        
        # Aggregate by token
        metrics = []
        for token in token_list:
            token_trades = df[df["mint"] == token]
            if token_trades.empty:
                continue
            
            total_in = token_trades[token_trades["direction"] == "in"]["value_usd"].sum()
            total_out = token_trades[token_trades["direction"] == "out"]["value_usd"].sum()
            net_cost = total_in - total_out
            
            # Current holdings
            net_amount = token_trades[token_trades["direction"] == "in"]["amount"].sum() - \
                         token_trades[token_trades["direction"] == "out"]["amount"].sum()
            current_price = await get_current_price(session, token)
            current_value = net_amount * current_price
            
            unrealized_pnl = current_value - net_cost
            realized_pnl = total_out - total_in
            total_pnl = unrealized_pnl + realized_pnl
            roi = (total_pnl / total_in * 100) if total_in > 0 else 0
            
            metrics.append({
                "wallet": wallet,
                "token": token,
                "net_amount": net_amount,
                "current_value": current_value,
                "realized_pnl": realized_pnl,
                "unrealized_pnl": unrealized_pnl,
                "total_pnl": total_pnl,
                "roi": roi,
                "in_profit": 1 if unrealized_pnl > 0 else 0
            })
        
        df_metrics = pd.DataFrame(metrics)
        portfolio_value = df_metrics["current_value"].sum()
        realized = df_metrics["realized_pnl"].sum()
        unrealized = df_metrics["unrealized_pnl"].sum()
        total_pnl = realized + unrealized
        roi = (total_pnl / (portfolio_value - total_pnl) * 100) if (portfolio_value - total_pnl) > 0 else 0
        
        summary = {
            "wallet": wallet,
            "portfolio_value": portfolio_value,
            "realized_pnl": realized,
            "unrealized_pnl": unrealized,
            "total_pnl": total_pnl,
            "roi": roi,
            "num_tokens_in_profit": df_metrics["in_profit"].sum()
        }
        return pd.DataFrame([summary])

# %% [markdown]
# ## Run Analysis for Multiple Wallets

# %%
async def analyze_traders(wallets, token_list):
    results = await asyncio.gather(*[analyze_wallet(w, token_list) for w in wallets])
    df = pd.concat(results, ignore_index=True)
    return df.sort_values("roi", ascending=False)

# %% [markdown]
# ### Example Run

# %%
wallets = ["FG3mMHRCToZyd77dM8bNstW3PzUsHNj4UwPrUzeFrzJ1", "3vKdbZPvuMgi9gCfKxBtKN2iMxfHpMy8haXWGe2AHuZ3"]
tokens = ["So11111111111111111111111111111111111111112", "9VzamStgtAgFErdWszYq3RTbcpAz7BeDnpYeMsCpYLT5"]

df = await analyze_traders(wallets, tokens)
print(df)


C:\Python310\lib\genericpath.py:77: RuntimeWarning: coroutine 'analyze_traders' was never awaited
  m = tuple(map(os.fspath, m))


AttributeError: 'str' object has no attribute 'get'

In [9]:
ana = await analyze_traders_for_tokens(["9VzamStgtAgFErdWszYq3RTbcpAz7BeDnpYeMsCpYLT5"])

KeyError: 'blocktime'

In [ ]:

# === CLI Test ===
if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--tokens", nargs="+", help="Token mint addresses (max 3)", required=False)
    parser.add_argument("--wallets", nargs="+", help="Wallet addresses to analyze", required=False)
    args = parser.parse_args()
    async def main():
        async with HttpClient() as client:
            toks = args.tokens or ["9VzamStgtAgFErdWszYq3RTbcpAz7BeDnpYeMsCpYLT5"]
            wallets = args.wallets or []
            print("Fetching trades...")
            df = await get_trades_with_prices(client, toks, limit=100)
            print("Trades rows:", 0 if df is None else len(df.index))
            if not df.empty:
                print(df.head(3))
            if wallets:
                net = await get_wallet_networth_with_trades(client, wallets, toks, view="focused", selected_tokens=toks)
                for w in net:
                    print(w["wallet_address"], w["total_value_usd"])
    asyncio.run(main())



In [ ]:
metric = await analyze_traders_for_tokens(['BGifmxmrU7yyUxhRhTQWt52qbB1PGERFiPYMrauDAo3z'])

AttributeError: 'list' object has no attribute 'empty'